## Multiplayer Proxy Failure _ Data analysis


In [1]:

import module
import numpy as np
import csv
from joblib import Parallel, delayed
from itertools import product
import pandas as pd


In [5]:
# use pandas to read filtered data
df = pd.read_csv('filtered_data_file.csv')


df["goal_improvement"] = df["maxed_goal_value"] - df["goal_base"]
df.describe()

,n,trials,proxy_discard,goal_discard,measure_time,spectral_radius,alpha,avg_number_of_edges,correlation_std,maxed_goal_value,goal_base,maxed_proxy_value,correlation_on_max_proxy,correlation_base,optimal_goal_value,optimal_proxy_value,correlation_on_optimal_proxy,proxy_base,goal_improvement
count,4260.000000,4260.0,4260.0,4260.000000,4260.0,4260.000000,4260.0,4260.000000,4260.000000,4260.000000,4.260000e+03,4.260000e+03,3351.000000,4260.000000,4.260000e+03,4.260000e+03,3416.000000,4.260000e+03,4260.000000
mean,633.241315,50.0,50.0,211.995305,50.0,1.859624,0.1,3.750000,0.373308,0.144581,1.269719e-04,2.181247e-01,0.581627,0.833771,8.459171e-02,1.993605e-01,0.374499,7.429088e-04,0.144454
std,571.816043,0.0,0.0,146.667641,0.0,0.732638,0.0,2.681266,0.123126,0.190042,3.797637e-02,2.175803e-01,0.399254,0.177018,1.951209e-01,2.092773e-01,0.546594,5.989310e-02,0.193729
min,120.000000,50.0,50.0,50.000000,50.0,1.000000,0.1,1.000000,0.102132,-0.218081,-2.375521e-01,2.038105e-12,-1.000000,0.212508,-9.551224e-01,5.460573e-12,-1.000000,-3.387972e-01,-0.277724
25%,256.000000,50.0,50.0,50.000000,50.0,1.000000,0.1,1.750000,0.287442,0.000107,-5.968582e-03,3.287826e-03,0.405930,0.737460,1.388775e-19,3.112627e-03,-0.022382,-1.045465e-02,0.000175
50%,512.000000,50.0,50.0,150.000000,50.0,1.500000,0.1,3.000000,0.362754,0.043763,-5.505714e-23,1.780743e-01,0.687120,0.881302,2.141998e-03,1.436812e-01,0.346297,1.192622e-20,0.043051
75%,1024.000000,50.0,50.0,250.000000,50.0,2.000000,0.1,5.000000,0.447971,0.254257,5.373771e-03,3.838415e-01,0.863717,1.000000,1.536771e-01,3.557232e-01,1.000000,1.302264e-02,0.249260
max,2048.000000,50.0,50.0,450.000000,50.0,3.000000,0.1,8.000000,0.882070,0.999947,2.873876e-01,9.911895e-01,1.000000,1.000000,9.999467e-01,9.939411e-01,1.000000,2.966129e-01,1.047842


In [13]:
import statsmodels.formula.api as smf


In [19]:
model = smf.ols("goal_improvement ~ n + proxy_discard + spectral_radius + alpha + avg_number_of_edges",
                    df).fit()

model.summary()
model.params

Intercept             -0.000026
n                     -0.000002
proxy_discard         -0.001305
spectral_radius        0.089953
alpha                 -0.000003
avg_number_of_edges    0.011614
dtype: float64